- 使用Gradio + colab打造一個簡單的網頁


In [ ]:
!pip install gradio

In [ ]:
# 使用gradio做一個計算圓周長的程式，輸入項目為半徑

import gradio as gr
import math

def calculate_circumference(radius):
  return 2 * math.pi * radius

iface = gr.Interface(
    fn=calculate_circumference,
    inputs=gr.Number(label="Radius"),
    outputs="number",
    title="Circumference Calculator",
    description="Calculate the circumference of a circle given its radius."
)

iface.launch()


In [ ]:
# 使用gradio做一個計算bmi指數的程式
import gradio as gr

def calculate_bmi(height, weight):
  """Calculates BMI based on height and weight."""
  try:
    height_meters = float(height) / 100  # Convert height from cm to meters
    bmi = float(weight) / (height_meters ** 2)
    return round(bmi, 2)
  except ValueError:
    return "Invalid input. Please enter numeric values."

iface = gr.Interface(
    fn=calculate_bmi,
    inputs=[
        gr.Number(label="Height (cm)"),
        gr.Number(label="Weight (kg)")
    ],
    outputs=gr.Textbox(label="BMI"),
    title="BMI Calculator",
    description="Calculate your BMI based on height and weight.",
)

iface.launch()


In [ ]:
#TODO: 試著使用gradio做一個計算圓面積的程式，輸入為半徑，輸出為面積

## 開始進行地圖跟Gradio的整合

- 資料來源:
- https://data.gov.tw/dataset/40448

In [ ]:
url = "https://data.moenv.gov.tw/api/v2/aqx_p_432?api_key=9e565f9a-84dd-4e79-9097-d403cae1ea75&limit=1000&sort=ImportDate%20desc&format=JSON"

response = requests.get(url)
response.raise_for_status()
data = response.json()['records']
df = pd.DataFrame(data)

In [ ]:
df.head()

,sitename,county,aqi,pollutant,status,so2,co,o3,o3_8hr,pm10,...,wind_speed,wind_direc,publishtime,co_8hr,pm2.5_avg,pm10_avg,so2_avg,longitude,latitude,siteid
0,基隆,基隆市,38,,良好,1.4,0.23,59,36,15,...,1.1,85,2025/05/13 09:00:00,0.2,9.3,20,1,121.760056,25.129167,1
1,汐止,新北市,48,,良好,0.8,0.28,53,26,11,...,0.5,308,2025/05/13 09:00:00,0.2,11.9,18,0,121.64081,25.06624,2
2,新店,新北市,38,,良好,0.6,0.36,41,41,9,...,1.1,312,2025/05/13 09:00:00,0.2,8.9,14,0,121.537778,24.977222,4
3,土城,新北市,61,細懸浮微粒,普通,1.2,0.56,37,31,23,...,0.9,84,2025/05/13 09:00:00,0.3,16.2,24,0,121.451861,24.982528,5
4,板橋,新北市,53,細懸浮微粒,普通,1.5,0.64,38,37,14,...,1,291,2025/05/13 09:00:00,0.3,13.2,20,1,121.458667,25.012972,6


In [ ]:
url = "https://data.moenv.gov.tw/api/v2/aqx_p_432?api_key=9e565f9a-84dd-4e79-9097-d403cae1ea75&limit=1000&sort=ImportDate%20desc&format=JSON"

response = requests.get(url)
response.raise_for_status()
data = response.json()['records']
df = pd.DataFrame(data)

# 假設您的資料包含 'sitename', 'longitude', 'latitude' 和 'pm2.5' 欄位
if 'sitename' not in df.columns or 'longitude' not in df.columns or 'latitude' not in df.columns or 'pm2.5' not in df.columns:
    print ("資料中缺少必要的測站名稱、經緯度或 pm2.5 欄位。")

# 將經緯度和 pm2.5 轉換為數值
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df['pm2.5'] = pd.to_numeric(df['pm2.5'], errors='coerce')

# 移除經緯度或 pm2.5 為 NaN 的列
df_cleaned = df.dropna(subset=['latitude', 'longitude', 'pm2.5'])

if df_cleaned.empty:
    print ("沒有有效的 pm2.5 資料點可以顯示。")


In [ ]:
# 如果只需要看部份資訊
my_title_list = ['publishtime', 'latitude', 'longitude', 'pm2.5']
df_cleaned[my_title_list].head()

,publishtime,latitude,longitude,pm2.5
0,2025/05/13 09:00:00,25.129167,121.760056,5
1,2025/05/13 09:00:00,25.066240,121.640810,8
2,2025/05/13 09:00:00,24.977222,121.537778,6
3,2025/05/13 09:00:00,24.982528,121.451861,17
4,2025/05/13 09:00:00,25.012972,121.458667,10


In [ ]:
center_lat = df_cleaned['latitude'].mean()
center_lon = df_cleaned['longitude'].mean()

In [ ]:
# 創建 Folium 地圖
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

In [ ]:
m

In [ ]:
# 根據 pm2.5 值添加標記
for index, row in df_cleaned.iterrows():
    #print(index, '-> ', row)
    station_name = row['sitename']
    latitude = row['latitude']
    longitude = row['longitude']
    pm25_value = row['pm2.5']

    color = 'green'
    if pm25_value > 35:
        color = 'orange'
    if pm25_value > 75:
        color = 'red'

    folium.CircleMarker(
        location=[latitude, longitude],
        radius=8,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        tooltip=f"{station_name}: PM2.5 = {pm25_value}"
    ).add_to(m)



In [ ]:
m

### 要使用地圖跟gradio做整合前，需要把處理資料的 Function 製作出來

In [ ]:
import gradio as gr
import math
import requests
import folium
import pandas as pd

# 載入環境資料
url = "https://data.moenv.gov.tw/api/v2/aqx_p_432?api_key=9e565f9a-84dd-4e79-9097-d403cae1ea75&limit=100&sort=ImportDate%20desc&format=JSON"

def visualize_pm25_marker():
    """
    抓取新的 PM2.5 資料並使用 Folium 視覺化 (使用一般的 Marker)。
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()['records']
        df = pd.DataFrame(data)

        # 假設您的資料包含 'sitename', 'longitude', 'latitude' 和 'pm2.5' 欄位
        if 'sitename' not in df.columns or 'longitude' not in df.columns or 'latitude' not in df.columns or 'pm2.5' not in df.columns:
            return "資料中缺少必要的測站名稱、經緯度或 pm2.5 欄位。"

        # 將經緯度和 pm2.5 轉換為數值
        df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
        df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
        df['pm2.5'] = pd.to_numeric(df['pm2.5'], errors='coerce')

        # 移除經緯度或 pm2.5 為 NaN 的列
        df_cleaned = df.dropna(subset=['latitude', 'longitude', 'pm2.5'])

        if df_cleaned.empty:
            return "沒有有效的 pm2.5 資料點可以顯示。"

        # 計算地圖的中心點
        center_lat = df_cleaned['latitude'].mean()
        center_lon = df_cleaned['longitude'].mean()

        # 創建 Folium 地圖
        m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

        # 根據 pm2.5 值添加標記 (使用 Marker)
        for index, row in df_cleaned.iterrows():
            station_name = row['sitename']
            latitude = row['latitude']
            longitude = row['longitude']
            pm25_value = row['pm2.5']

            # 根據 PM2.5 值決定 Marker 的顏色 (可以調整 icon 或顏色)
            icon_color = 'green'
            if pm25_value > 35:
                icon_color = 'orange'
            if pm25_value > 75:
                icon_color = 'red'

            folium.Marker(
                location=[latitude, longitude],
                tooltip=f"{station_name}: PM2.5 = {pm25_value}",
                icon=folium.Icon(color=icon_color)
            ).add_to(m)

        # 將 Folium 地圖物件轉換為 HTML 字串
        map_html = m._repr_html_()
        return map_html
    except Exception as e:
        return f"發生未預期的錯誤：{e}"

iface_marker = gr.Interface(
    fn=visualize_pm25_marker,
    inputs=None,
    outputs=gr.HTML(label="PM2.5 測站分佈地圖"),
    title="PM2.5 測站資料視覺化 (Marker)"
)

iface_marker.launch()


In [ ]:
# 使用另一種 marker

import requests
import folium
import gradio as gr
import pandas as pd

def visualize_pm25_v4():
    url = "https://data.moenv.gov.tw/api/v2/aqx_p_432?api_key=9e565f9a-84dd-4e79-9097-d403cae1ea75&limit=100&sort=ImportDate%20desc&format=JSON"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()['records']
        df = pd.DataFrame(data)

        # 假設您的資料包含 'sitename', 'longitude', 'latitude' 和 'pm2.5' 欄位
        if 'sitename' not in df.columns or 'longitude' not in df.columns or 'latitude' not in df.columns or 'pm2.5' not in df.columns:
            return "資料中缺少必要的測站名稱、經緯度或 pm2.5 欄位。"

        # 將經緯度和 pm2.5 轉換為數值
        df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
        df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
        df['pm2.5'] = pd.to_numeric(df['pm2.5'], errors='coerce')

        # 移除經緯度或 pm2.5 為 NaN 的列
        df_cleaned = df.dropna(subset=['latitude', 'longitude', 'pm2.5'])

        if df_cleaned.empty:
            return "沒有有效的 pm2.5 資料點可以顯示。"

        # 計算地圖的中心點
        center_lat = df_cleaned['latitude'].mean()
        center_lon = df_cleaned['longitude'].mean()

        # 創建 Folium 地圖
        m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

        # 根據 pm2.5 值添加標記
        for index, row in df_cleaned.iterrows():
            station_name = row['sitename']
            latitude = row['latitude']
            longitude = row['longitude']
            pm25_value = row['pm2.5']

            color = 'green'
            if pm25_value > 35:
                color = 'orange'
            if pm25_value > 75:
                color = 'red'

            folium.CircleMarker(
                location=[latitude, longitude],
                radius=8,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.7,
                tooltip=f"{station_name}: PM2.5 = {pm25_value}"
            ).add_to(m)

        # *** 重要：將 Folium 地圖物件轉換為 HTML 字串 ***
        map_html = m._repr_html_()
        return map_html
    except Exception as e:
        return f"發生未預期的錯誤：{e}"

iface_v4 = gr.Interface(
    fn=visualize_pm25_v4,
    inputs=None,
    outputs=gr.HTML(label="PM2.5 測站分佈地圖"),
    title="PM2.5 測站資料視覺化",
)

iface_v4.launch()